In [1]:
library(tidyverse)

── Attaching packages ─────────────────────────────────────── tidyverse 1.2.1 ──
✔ ggplot2 2.2.1     ✔ purrr   0.2.4
✔ tibble  1.4.2     ✔ dplyr   0.7.4
✔ tidyr   0.8.0     ✔ stringr 1.2.0
✔ readr   1.1.1     ✔ forcats 0.2.0
── Conflicts ────────────────────────────────────────── tidyverse_conflicts() ──
✖ dplyr::filter() masks stats::filter()
✖ dplyr::lag()    masks stats::lag()


In [2]:
data_dir <- file.path("..", "data")

In [3]:
model <- readRDS(file.path(data_dir, "multivariate_model.rds"))

In [4]:
coefs_mtx <- confint(model)

Waiting for profiling to be done...


In [5]:
coefs <- coefs_mtx %>%
exp %>%
as.data.frame(stringsAsFactors = FALSE) %>%
setNames(c("lo", "hi")) %>%
mutate(cov = rownames(.))

ests <- model %>%
summary %>%
pluck("coefficients") %>%
as.data.frame(stringsAsFactors = FALSE) %>%
setNames(c("est", "se", "zval", "pval")) %>%
mutate(cov = rownames(.)) %>%
mutate(est = exp(est))

df <- coefs %>%
full_join(ests, by = "cov") %>%
mutate(ests = sprintf("%s (%s~%s)",
                      round(est, 2), round(lo, 2), round(hi, 2))) %>%
mutate(cov_ = cov) %>%
mutate(cov = case_when(
    grepl("icu_adm_hour|po2|pco2", cov) ~ cov,
    TRUE ~ str_replace_all(cov, "[0-9]", "")
)) %>%
mutate(cov = str_replace_all(cov, "icu_adm_weekday|icu_adm_hour", "")) %>%
mutate(cov = str_replace_all(cov, "_", " ")) %>%
mutate(cov = str_replace_all(cov, "lab|vs|flag|icd|first", "")) %>%
mutate(cov = str_replace_all(cov, "\\s+$|^\\s+", "")) %>%
mutate(cov = case_when(
    grepl("icd|lab|day", cov_) ~ toupper(cov),
    TRUE ~ tools::toTitleCase(cov)
)) %>%
mutate(cov = case_when(
    cov == "Echo" ~ "TTE",
    cov == "genderF" ~ "Gender (Female)",
    cov == "careunitMICU" ~ "Service Unit (MICU)",
    cov == "Weight" ~ "Weight (Kg)",
    cov == "Elix Score" ~ "Elixhauser Score",
    cov == "Saps" ~ "SAPS Score",
    cov == "Sofa" ~ "SOFA Score",
    cov == "Vent" ~ "Mechanical Ventilation (1st 24 Hr)",
    cov == "Vaso" ~ "Vasopressor (1st 24 Hr)",
    cov == "Sedative" ~ "Sedative (1st 24 Hr)",
    cov == "Icu Adm Weekday" ~ "Day of ICU Admission",
    cov == "Icu Adm Hour" ~ "Hour of ICU Admission",
    cov == "Temp" ~ "Temperature (C)",
    cov == "Map" ~ "MAP",
    cov == "Cvp" ~ "CVP (measured)",
    cov %in% c("TROPONIN", "BNP", "CREATININE KINASE") ~ paste(cov, "(tested)"),
    TRUE ~ cov
)) %>%
filter(!grepl("Intercept", cov)) %>%
mutate(pval = case_when(
    pval < 0.01 ~ "<0.01",
    TRUE ~ as.character(round(pval, 2))
)) %>%
select(cov, ests, pval) %>%
setNames(c("Covariate", "Odds Ratio (2.5%~97.5%)", "p value"))

df

Covariate,Odds Ratio (2.5%~97.5%),p value
TTE,0.65 (0.53~0.79),<0.01
Service Unit (MICU),1.39 (1.09~1.79),<0.01
Age,1.02 (1.01~1.03),<0.01
Gender (Female),0.85 (0.7~1.03),0.11
Weight (Kg),0.99 (0.99~1),<0.01
SAPS Score,1.06 (1.03~1.09),<0.01
SOFA Score,1.18 (1.14~1.23),<0.01
Elixhauser Score,1.03 (1.01~1.05),<0.01
Mechanical Ventilation (1st 24 Hr),1.32 (0.95~1.82),0.1
Vasopressor (1st 24 Hr),0.98 (0.77~1.24),0.86


In [6]:
data.table::fwrite(df, file.path(data_dir, "coefficients.csv"))